In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai import *
from fastai.vision import *
from fastai.callbacks import *

In [4]:
PATH = Path('/content/data/')

In [6]:
np.random.seed(42)

In [5]:
src = (ImageList.from_csv??

Signature: ImageList.from_csv(path:Union[pathlib.Path, str], csv_name:str, header:str='infer', delimiter:str=None, **kwargs) -> 'ItemList'
Source:   
    @classmethod
    def from_csv(cls, path:PathOrStr, csv_name:str, header:str='infer', delimiter:str=None, **kwargs)->'ItemList':
        "Get the filenames in `path/csv_name` opened with `header`."
        path = Path(path)
        df = pd.read_csv(path/csv_name, header=header, delimiter=delimiter)
        return cls.from_df(df, path=path, **kwargs)
File:      /usr/local/lib/python3.6/dist-packages/fastai/vision/data.py
Type:      method


In [9]:
src = (ImageList.from_csv(PATH,'train.csv',folder='train',suffix='.jpg')
       .
       .label_from_df()
      )

Exception: Your data isn't split, if you don't want a validation set, please use `split_none`.

In [8]:
src

ImageList (18540 items)
Image (3, 500, 500),Image (3, 500, 500),Image (3, 500, 500),Image (3, 500, 500),Image (3, 500, 500)
Path: /content/data